In [ ]:
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import math

ORBIT_DATA_DIR  = Path(r"E:\interim\step2_normalized_test")
OB_MEET_TIME    = Path(r"E:\tables\orbit_meet_time_condition_test.csv")
OUTPUT_DIR      = Path(r"E:\tables")

print("orbit data directory =", {ORBIT_DATA_DIR})
print("Orbit_meet_time_condion.csv path=", {OB_MEET_TIME})
print("Directory of output", {OUTPUT_DIR})

a = 6378137                                                         # the long radius of the Earth
b = 6356752.314                                                     # the short radius of the Earth

e = math.sqrt((a*a - b*b)/(a*a))                                    # eccentricity of the Earth

df_obmt = pd.read_csv(OB_MEET_TIME)

length_omt = len(df_obmt)

list1 = [[] for k in range(length_omt)]



In [ ]:
for i in range(length_omt):
    list2 = []
    orbit_file_name = df_obmt["orbit_file"].iloc[i]
    
    orbit_data_path = ORBIT_DATA_DIR / orbit_file_name

    df_obdata = pd.read_csv(orbit_data_path)

    
    for j in range(len(df_obdata)):
        lat1 = df_obmt["eq_lat"].iloc[i] * math.pi/180
        lon1 = df_obmt["eq_lon"].iloc[i] * math.pi/180

        lat2 = df_obdata["lat"].iloc[j] * math.pi/180
        lon2 = df_obdata["lon"].iloc[j] * math.pi/180

        dif_lat = lat2 - lat1
        dif_lon = lon2 - lon1
        if dif_lon > math.pi:
            dif_lon -= 2*math.pi
        elif dif_lon < -math.pi:
            dif_lon += 2*math.pi
        
        P = (lat1+lat2) / 2                                         # 両点緯度の平均値
        W = math.sqrt((1-e*e * math.sin(P) * math.sin(P)))
        M = (a*(1 - e*e)) / (W * W * W)
        N = a / W

        dist = math.sqrt(dif_lat*dif_lat*M*M + dif_lon*dif_lon*N*N*math.cos(P)*math.cos(P))
        dist = dist / 1000                                          # 単位[km] => [m]

        if dist < 330:
            beforeq = pd.to_datetime(df_obmt["4hour_before"].iloc[i]).tz_localize(None)
            print("beforeq", beforeq, "type=", type(beforeq))
            starteq = pd.to_datetime(df_obmt["datetime"].iloc[i]).tz_localize(None)
            print("starteq", starteq, "type=", type(starteq))
            s1 = pd.to_datetime(df_obdata["datetime"].iloc[j])
            print("s1", s1, "type=", type(s1))

            if beforeq <= s1 <= starteq:
                list2.append(orbit_file_name)
                break
    
    print(list2)
    list1[i] = list2


In [ ]:
print(list1)
data = pd.DataFrame(list1)
output_one = pd.concat([data, df_obmt], axis=1)
print(output_one)
